In [11]:
import datetime
import csv
import time
import smtplib
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [ ]:
def get_page_soup(url, headers):
    response = requests.get(url, headers=headers)
    return BeautifulSoup(response.content, "html.parser")

def get_product_info(soup):
    title = soup.find(id='productTitle').get_text().strip()
    price = soup.find(id='priceblock_ourprice').get_text().strip()[1:]
    return title, price

def write_to_csv(filename, data):
    header = ['Title', 'Price', 'Date']
    with open(filename, 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerow(data)

def append_to_csv(filename, data):
    with open(filename, 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)

def check_price(url, headers, filename):
    soup = get_page_soup(url, headers)
    title, price = get_product_info(soup)
    today = datetime.date.today()
    data = [title, price, today]
    write_to_csv(filename, data)

In [ ]:

# Set up constants
URL = 'https://www.amazon.com/Gildan-V-Neck-T-Shirts-Multipacks-Black/dp/B07JBRCX8X/ref=sr_1_42?crid=4JPH4XVBHQ0Q&keywords=t-shirt&qid=1688730957&sprefix=t+shir%2Caps%2C400&sr=8-42'
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
    "Accept-Encoding": "gzip, deflate",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT": "1",
    "Connection": "close",
    "Upgrade-Insecure-Requests": "1"
}
FILENAME = 'AmazonWebScraperDataset.csv'

In [ ]:
# Run check_price after a set time and append data to the CSV
while True:
    check_price(URL, HEADERS, FILENAME)
    time.sleep(86400)

# Read CSV into DataFrame
df = pd.read_csv('AmazonWebScraperDataset.csv')
print(df)


In [ ]:

# Send an email when the price hits below a certain level
def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login('halimmansour99@gmail.com', 'xxxxxxxxxxxxxx')

    subject = "The Shirt you want is below $15! Now is your chance to buy!"
    body = "Halim, This is the moment we have been waiting for. Now is your chance to pick up the shirt of your dreams. Don't mess it up! Link here: https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ/ref=sr_1_3?dchild=1&keywords=data+analyst+tshirt&qid=1626655184&sr=8-3"

    msg = f"Subject: {subject}\n\n{body}"

    server.sendmail('halimmansour99@gmail.com', msg)

# Call send_mail function if the price drops below a certain level
# (add the necessary conditions and logic to trigger the email)
if price < 15:
    send_mail()